In [1]:
import pandas as pd

## Lets explore indicators and find something about human population of particular ages
I skip some intermediate results until I found **«Population ages 65 and above (% of total)»**

In [2]:
indicators = pd.read_csv('../input/world-development-indicators/Series.csv')
# Statement below allows you to show all content of a cell in a row 
pd.set_option('display.max_colwidth', -1)
# Get structure
#indicators.head()
# Explore related
print(indicators.shape)
pop_mask = indicators['IndicatorName'].str.contains('population age', case=False)
indicators[pop_mask][['SeriesCode','IndicatorName', 'LongDefinition']].sort_values(by=['IndicatorName'])

## Will be working further with this one "Population ages 65 and above (% of total)"
LongDefinition contains an explanation of above

In [3]:
indicators[indicators['SeriesCode']=='SP.POP.65UP.TO.ZS'][['IndicatorName', 'LongDefinition']].sort_values(by=['IndicatorName'])

## Load indicators with values into the dataframe

In [4]:
# Fill in dataframe
data = pd.read_csv('../input/world-development-indicators/Indicators.csv')

## See the structure

In [5]:
print(pd.unique(data['CountryCode']).shape)
data.head()

## How it looks to get only Population ages 65 and above (% of total)

In [6]:
mask = data['IndicatorCode']=='SP.POP.65UP.TO.ZS'
data[mask].head()

## Slice data and pivot dataframe to be able use builtin corellation function

In [7]:
pivot_data = data[mask].pivot(index='Year', columns='CountryName', values='Value')
print(pivot_data.shape)
pivot_data.head()

## Do clean up from NaN values

In [8]:
pivot_data = pivot_data.dropna(axis=1, how='any')
print(pivot_data.shape)

## less 3 columns after clean up above 228 -> 225

## Get corellation of each country with World

In [9]:
corr_pivot_data = pivot_data.corr()
corr_pivot_data[['World']].sort_values('World')

## Lets choose following countries for further research

In [10]:
res_country_list = ['World', 'Ukraine', 'United States', 'Angola', 'Chad']

## Coose and slice interesting countries

In [11]:
pivot_data_extract = pivot_data[res_country_list]
pivot_data_extract.head()

## Plot them to find out does life gets longer and longer around the world

In [12]:
axes = pivot_data_extract.plot(title='Population ages 65 and above (% of total)',
                        grid=True, figsize=[12,4])
axes.set_ylabel('(% of total)')
# To start axes with right numbers
axes.axis([1959,2015,0,17])


# Here I understood Population ages 65 and above is not enough
## and star using the other indicators

## Indicator Life expectancy

In [13]:
pop_mask = indicators['IndicatorName'].str.contains('life exp.*total', case=False)
indicators[pop_mask][['SeriesCode','IndicatorName', 'LongDefinition']].sort_values(by=['IndicatorName'])

In [14]:
lifeexp = data[data['IndicatorCode']=='SP.DYN.LE00.IN']
# lifeexp.head()
lifeexp_pivot = lifeexp.pivot(index='Year', columns='CountryName', values='Value')
# lifeexp_pivot.head()
print('Original ', lifeexp_pivot.shape)
lifeexp_pivot = lifeexp_pivot.dropna(axis=1, how='any')
print('After dropna()', lifeexp_pivot.shape)
axes = lifeexp_pivot[res_country_list].plot(title='Life expectancy at birth',
                        grid=True, figsize=[12,4])
axes.set_ylabel('age')
# To start axes with right numbers
axes.axis([1959,2015,0,80])

# Here is another dataset United Nations Population Indicators
## Contains only Median age by Country and Years by 5 1950-2015

In [15]:
# Use statements below to check files location
#from subprocess import check_output
#print(check_output(["ls", "../input/unated-nations-pop-median-age"]).decode("utf8"))
stage = pd.read_csv('../input/unated-nations-pop-median-age/WPP2017_POP_F05_MEDIAN_AGE.csv')
print(stage.shape)
stage.head()

## Unpivot to the usable shape, same as for above research
### Data is clean from scratch

In [16]:
stage_unpivot = pd.melt(stage, id_vars=['Year'], value_name='Value')
stage_unpivot = stage_unpivot.rename(index=str, columns={'Year':'CountryName', 'variable':'Year'})
stage_unpivot.head()
popmedianage_pivot = stage_unpivot.pivot(index='Year', columns='CountryName', values='Value')
popmedianage_pivot

In [17]:
axes = popmedianage_pivot[['WORLD', 'Ukraine', 'United States of America', 'Angola', 'Chad']].plot(grid=True, figsize=[12,4])
axes.set_title('Population median age by Country since 1950')
axes.set_ylabel('Median age')
#axes.axis([1950,2020,0,100])

## Please find findings in presentation